# はじめに

急にムラムラしてきたので、xyY の Color Volume をプロットしてみる。<br>
初の3次元プロットだけど上手くいくかしら？

# やり方

大きく2通りある。

1. R, G, B を 0～255 まで振って $2^{24}$ 通りの xyY をプロットする
2. gamut 内部を 100x100 くらいでサンプリングして、各点でYが最大となるRGBの組を求め、それを xyY に変換する

1.のやり方を「全網羅」、2.のやり方を「外面」と呼ぶことにする。

# 全網羅 方式

0～255 はあまり意味ないので、0～1 でやりましょう。

```
r = np.linspace(0, 1, 256)
g = np.linspace(0, 1, 256)
b = np.linspace(0, 1, 256)
```

的な感じで。

# 外面 方式

こちらは少々面倒くさい。

1. xy 平面上を 100x100 くらいでサンプリングする。
2. 各点が gamut の領域内か判定。領域内なら配列に入れる。
3. 領域内の xy ペア を $Y=1$で xyY --> RGBに変換。この時、RGBが1.0 を超えた場合は max が 1.0 となるよう正規化する。
4. 正規化した RGB 値を xyY に変換。そしてプロット。

# で、どれを実装するの？

面白そうな外面方式を組んで見る。

In [1]:

%matplotlib inline
import plor_color_gamut as pcg
import imp
imp.reload(pcg)
pcg

<module 'plor_color_gamut' from 'C:\\home\\sip\\matplotlib\\color_volume\\plor_color_gamut.py'>